In [101]:
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from os.path import join
import pandas as pd
from utilities import utils
from stats_tools import vis as svis
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 40
plt.style.use('ggplot')
import importlib
importlib.reload(svis)
from pydicom import dcmread

In [ ]:
dcx

In [97]:
sif_dir = 'Y:/'

In [67]:
fig_dir = join(base_dir, 'figs')
table_dir = join(base_dir, 'data/tables')
dfc = utils.load_scan_csv(join(table_dir, 'neg_pos_clean.csv'))
df_all = utils.load_scan_csv(join(table_dir, 'neg_pos.csv'))

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


### Directories on sif

In [90]:
df_sif_dir = utils.load_scan_csv(join(table_dir, 'series_directories.csv'))
sif_dir_dic= pd.Series(df_sif_dir.Directory.values, index=df_sif_dir.SeriesInstanceUID).to_dict()

In [79]:
df_all3dt1 = df_all[(df_all.Sequence=='t1') & (df_all.MRAcquisitionType=='3D')]

In [84]:
df_all3dt1_missing_date = df_all3dt1[df_all3dt1.InstanceCreationDate.isna()]

In [103]:
for sid in df_all3dt1_missing_date.SeriesInstanceUID[:2]:
    series_dir = join(sif_dir, sif_dir_dic[sid])
    dcm_name = os.listdir(series_dir)[0]
    dcm = dcmread(join(series_dir, dcm_name))
    

## Get 3d T1 scan pairs

In [9]:
df_3dt1 = dfc[(dfc.Sequence=='t1') & (dfc.MRAcquisitionType=='3D') & (dfc.NumberOfSlices>=64)]
print(len(df_3dt1), len(df_3dt1.PatientID.unique()))
gb_pat_date = df_3dt1.groupby('PatientID').InstanceCreationDate.nunique()
gb_masked = gb_pat_date[gb_pat_date>1]
pat_scan_pairs = list(gb_masked.index)
print(len(pat_scan_pairs))
df_3dt1_pairs = df_3dt1[df_3dt1.PatientID.isin(pat_scan_pairs)]
print(len(df_3dt1_pairs))

35594 14005
1724
8711


In [11]:
mr_path = 'D:\F\\test\\b91f4014288ed99919b6e6b1483ac8a4\\3809675063ea32c46697dd74c0b25455.dcm'


In [12]:
dcm = dcmread(mr_path)

In [15]:
dcm

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 180
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 3809675063ea32c46697dd74c0b25455
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.9.3811.1.5.6
(0002, 0013) Implementation Version Name         SH: 'PYNETDICOM_156'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M_FFE', 'M', 'FFE']
(0008, 0012) Instance Creation Date              DA: '20190121'
(0008, 0013) Instance Creation Time              TM: '172453.050'
(0008, 0014) Instance Creator 

In [39]:
dcm[0x2005,0x140F][0]['AcquisitionDateTime'].value

'20190121'

In [48]:
dfc.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [64]:
dcm[0x2005, 0x140F][0]['MRAcquisitionPhaseEncodingStepsInPlane'].value
dcm[0x2005, 0x140F][0]['MRAcquisitionPhaseEncodingStepsOutOfPlane'].value
dcm[0x2005, 0x140F][0]['AcquisitionContrast'].value
dcm[0x2005, 0x140F][0]['PulseSequenceName'].value
dcm['PercentSampling'].value
dcm['PercentPhaseFieldOfView'].value
dcm['PixelBandwidth'].value
dcm['ReconstructionDiameter'].value

'281.0'

In [57]:
dcm[0x008,0x02A]

KeyError: (0008, 002a)